In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random

os.environ["CUDA_VISIBLE_DEVICES"] = '3'
torch.backends.cudnn.benchmark = True
NB_CLASS=59

In [2]:
BATCH_SIZE=32
IMAGE_SIZE=224    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.resnet101_finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model

In [3]:
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/ResNet101/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet101/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet101', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet101', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [4]:
train(60)

[+] loading model... Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.135417,loss:3.958422
[epoch:0,batch:59]:acc: 0.247917,loss:3.458655
[epoch:0,batch:89]:acc: 0.331597,loss:3.105424
[epoch:0,batch:119]:acc: 0.382031,loss:2.845823
[epoch:0,batch:149]:acc: 0.414375,loss:2.662219
[epoch:0,batch:179]:acc: 0.450347,loss:2.501971
[epoch:0,batch:209]:acc: 0.478571,loss:2.358621
[epoch:0,batch:239]:acc: 0.493620,loss:2.256741
[epoch:0,batch:269]:acc: 0.507292,loss:2.161558
[epoch:0,batch:299]:acc: 0.518333,loss:2.089002
[epoch:0,batch:299]: val_loss:1.251998,val_acc:0.638687,val_total:4539
[epoch:0,batch:329]:acc: 0.531250,loss:2.008029
[epoch:0,batch:359]:acc: 0.540799,loss:1.947246
[epoch:0,batch:389]:acc: 0.547756,loss:1.899240
[epoch:0,batch:419]:acc: 0.555580,loss:1.850565
[epoch:0,batch:449]:acc: 0.560764,loss:1.810975
[epoch:0,batch:479]:acc: 0.565820,loss:1.773251
[epoch:0,batch:509]:acc: 0.571691,loss:1.737486
[epoch:0,batch:539]:acc: 0.576273,loss:1.70330

[epoch:4,batch:119]:acc: 0.810677,loss:0.494143
[epoch:4,batch:149]:acc: 0.807917,loss:0.502843
[epoch:4,batch:179]:acc: 0.801389,loss:0.519436
[epoch:4,batch:209]:acc: 0.801339,loss:0.517142
[epoch:4,batch:239]:acc: 0.800521,loss:0.521532
[epoch:4,batch:269]:acc: 0.801157,loss:0.518860
[epoch:4,batch:299]:acc: 0.801042,loss:0.516883
[epoch:4,batch:299]: val_loss:0.473274,val_acc:0.823089,val_total:4539
[epoch:4,batch:329]:acc: 0.802083,loss:0.513340
[epoch:4,batch:359]:acc: 0.803472,loss:0.511589
[epoch:4,batch:389]:acc: 0.803446,loss:0.512596
[epoch:4,batch:419]:acc: 0.803720,loss:0.512181
[epoch:4,batch:449]:acc: 0.803681,loss:0.511886
[epoch:4,batch:479]:acc: 0.802930,loss:0.513160
[epoch:4,batch:509]:acc: 0.803738,loss:0.512643
[epoch:4,batch:539]:acc: 0.802604,loss:0.511485
[epoch:4,batch:569]:acc: 0.802741,loss:0.510872
[epoch:4,batch:599]:acc: 0.801823,loss:0.510388
[epoch:4,batch:599]: val_loss:0.466851,val_acc:0.815598,val_total:4539
[epoch:4,batch:629]:acc: 0.802381,loss:0.5

[epoch:8,batch:269]:acc: 0.843866,loss:0.374009
[epoch:8,batch:299]:acc: 0.843437,loss:0.371917
[epoch:8,batch:299]: val_loss:0.449825,val_acc:0.829258,val_total:4539
[epoch:8,batch:329]:acc: 0.843466,loss:0.376994
[epoch:8,batch:359]:acc: 0.843142,loss:0.377778
[epoch:8,batch:389]:acc: 0.845433,loss:0.375167
[epoch:8,batch:419]:acc: 0.846057,loss:0.373982
[epoch:8,batch:449]:acc: 0.846667,loss:0.374839
[epoch:8,batch:479]:acc: 0.847461,loss:0.374049
[epoch:8,batch:509]:acc: 0.847855,loss:0.372250
[epoch:8,batch:539]:acc: 0.848148,loss:0.372043
[epoch:8,batch:569]:acc: 0.848629,loss:0.368811
[epoch:8,batch:599]:acc: 0.848854,loss:0.368121
[epoch:8,batch:599]: val_loss:0.440093,val_acc:0.832562,val_total:4539
[epoch:8,batch:629]:acc: 0.848562,loss:0.369066
[epoch:8,batch:659]:acc: 0.849669,loss:0.367648
[epoch:8,batch:689]:acc: 0.849457,loss:0.367331
[epoch:8,batch:719]:acc: 0.849740,loss:0.366440
[epoch:8,batch:749]:acc: 0.849417,loss:0.367024
[epoch:8,batch:779]:acc: 0.848558,loss:0.3

[epoch:12,batch:299]: val_loss:0.378156,val_acc:0.861864,val_total:4539
[epoch:12,batch:329]:acc: 0.889489,loss:0.240983
[epoch:12,batch:359]:acc: 0.891233,loss:0.238129
[epoch:12,batch:389]:acc: 0.891026,loss:0.238433
[epoch:12,batch:419]:acc: 0.890551,loss:0.237937
[epoch:12,batch:449]:acc: 0.891597,loss:0.236787
[epoch:12,batch:479]:acc: 0.892122,loss:0.237861
[epoch:12,batch:509]:acc: 0.891850,loss:0.239053
[epoch:12,batch:539]:acc: 0.892361,loss:0.237967
[epoch:12,batch:569]:acc: 0.892434,loss:0.238081
[epoch:12,batch:599]:acc: 0.893125,loss:0.237290
[epoch:12,batch:599]: val_loss:0.380803,val_acc:0.862525,val_total:4539
[epoch:12,batch:629]:acc: 0.892758,loss:0.238091
[epoch:12,batch:659]:acc: 0.893324,loss:0.237515
[epoch:12,batch:689]:acc: 0.892844,loss:0.238349
[epoch:12,batch:719]:acc: 0.893142,loss:0.237816
[epoch:12,batch:749]:acc: 0.893708,loss:0.237438
[epoch:12,batch:779]:acc: 0.894191,loss:0.237753
[epoch:12,batch:809]:acc: 0.893981,loss:0.237713
[epoch:12,batch:839]:ac

[epoch:16,batch:389]:acc: 0.898077,loss:0.225505
[epoch:16,batch:419]:acc: 0.896652,loss:0.227463
[epoch:16,batch:449]:acc: 0.896875,loss:0.228193
[epoch:16,batch:479]:acc: 0.896094,loss:0.230303
[epoch:16,batch:509]:acc: 0.896324,loss:0.229352
[epoch:16,batch:539]:acc: 0.895660,loss:0.230764
[epoch:16,batch:569]:acc: 0.895833,loss:0.231461
[epoch:16,batch:599]:acc: 0.895833,loss:0.232476
[epoch:16,batch:599]: val_loss:0.371387,val_acc:0.868033,val_total:4539
[epoch:16,batch:629]:acc: 0.895337,loss:0.233162
[epoch:16,batch:659]:acc: 0.895218,loss:0.232859
[epoch:16,batch:689]:acc: 0.895924,loss:0.231167
[epoch:16,batch:719]:acc: 0.896398,loss:0.230501
[epoch:16,batch:749]:acc: 0.897167,loss:0.229299
[epoch:16,batch:779]:acc: 0.897516,loss:0.229530
[epoch:16,batch:809]:acc: 0.896991,loss:0.229979
[epoch:16,batch:839]:acc: 0.897693,loss:0.229785
[epoch:16,batch:869]:acc: 0.897522,loss:0.229342
[epoch:16,batch:899]:acc: 0.897500,loss:0.229522
[epoch:16,batch:899]: val_loss:0.369876,val_ac

...

[epoch:41,batch:179]:acc: 0.898090,loss:0.225383
[epoch:41,batch:209]:acc: 0.899702,loss:0.225003
[epoch:41,batch:239]:acc: 0.899089,loss:0.228562
[epoch:41,batch:269]:acc: 0.899769,loss:0.230331
[epoch:41,batch:299]:acc: 0.900833,loss:0.228327
[epoch:41,batch:299]: val_loss:0.368577,val_acc:0.869354,val_total:4539
[epoch:41,batch:329]:acc: 0.900852,loss:0.227690
[epoch:41,batch:359]:acc: 0.901997,loss:0.227389
[epoch:41,batch:389]:acc: 0.901362,loss:0.225863
[epoch:41,batch:419]:acc: 0.900223,loss:0.226644
[epoch:41,batch:449]:acc: 0.900486,loss:0.225476
[epoch:41,batch:479]:acc: 0.901432,loss:0.225103
[epoch:41,batch:509]:acc: 0.901225,loss:0.226930
[epoch:41,batch:539]:acc: 0.901042,loss:0.226598
[epoch:41,batch:569]:acc: 0.901042,loss:0.226550
[epoch:41,batch:599]:acc: 0.900260,loss:0.228124
[epoch:41,batch:599]: val_loss:0.371382,val_acc:0.869575,val_total:4539
[epoch:41,batch:629]:acc: 0.899702,loss:0.229461
[epoch:41,batch:659]:acc: 0.900047,loss:0.228846
[epoch:41,batch:689]:ac

[epoch:45,batch:299]: val_loss:0.369320,val_acc:0.865609,val_total:4539
[epoch:45,batch:329]:acc: 0.899148,loss:0.227984
[epoch:45,batch:359]:acc: 0.900434,loss:0.226994
[epoch:45,batch:389]:acc: 0.899679,loss:0.227330
[epoch:45,batch:419]:acc: 0.898363,loss:0.228983
[epoch:45,batch:449]:acc: 0.897569,loss:0.230233
[epoch:45,batch:479]:acc: 0.898372,loss:0.228914
[epoch:45,batch:509]:acc: 0.898775,loss:0.227618
[epoch:45,batch:539]:acc: 0.898495,loss:0.229360
[epoch:45,batch:569]:acc: 0.898849,loss:0.228698
[epoch:45,batch:599]:acc: 0.898906,loss:0.228306
[epoch:45,batch:599]: val_loss:0.369950,val_acc:0.868473,val_total:4539
[epoch:45,batch:629]:acc: 0.898909,loss:0.228559
[epoch:45,batch:659]:acc: 0.898674,loss:0.229320
[epoch:45,batch:689]:acc: 0.899230,loss:0.229259
[epoch:45,batch:719]:acc: 0.899306,loss:0.228634
[epoch:45,batch:749]:acc: 0.899500,loss:0.228081
[epoch:45,batch:779]:acc: 0.899038,loss:0.228839
[epoch:45,batch:809]:acc: 0.899383,loss:0.228875
[epoch:45,batch:839]:ac

[epoch:49,batch:449]:acc: 0.904375,loss:0.222941
[epoch:49,batch:479]:acc: 0.903711,loss:0.223947
[epoch:49,batch:509]:acc: 0.903922,loss:0.222901
[epoch:49,batch:539]:acc: 0.902836,loss:0.225117
[epoch:49,batch:569]:acc: 0.902851,loss:0.224521
[epoch:49,batch:599]:acc: 0.903073,loss:0.224290
[epoch:49,batch:599]: val_loss:0.369702,val_acc:0.865389,val_total:4539
[epoch:49,batch:629]:acc: 0.902927,loss:0.224490
[epoch:49,batch:659]:acc: 0.902415,loss:0.224498
[epoch:49,batch:689]:acc: 0.902264,loss:0.225080
[epoch:49,batch:719]:acc: 0.902214,loss:0.224849
[epoch:49,batch:749]:acc: 0.901917,loss:0.225726
[epoch:49,batch:779]:acc: 0.900801,loss:0.227770
[epoch:49,batch:809]:acc: 0.901003,loss:0.227427
[epoch:49,batch:839]:acc: 0.900781,loss:0.227505
[epoch:49,batch:869]:acc: 0.900467,loss:0.227401
[epoch:49,batch:899]:acc: 0.900243,loss:0.227385
[epoch:49,batch:899]: val_loss:0.369034,val_acc:0.867592,val_total:4539
[epoch:49,batch:929]:acc: 0.900605,loss:0.226989
[epoch:49,batch:959]:ac

[epoch:53,batch:539]:acc: 0.895718,loss:0.231808
[epoch:53,batch:569]:acc: 0.895888,loss:0.231378
[epoch:53,batch:599]:acc: 0.896927,loss:0.229557
[epoch:53,batch:599]: val_loss:0.372709,val_acc:0.869354,val_total:4539
[epoch:53,batch:629]:acc: 0.897123,loss:0.229082
[epoch:53,batch:659]:acc: 0.897822,loss:0.227519
[epoch:53,batch:689]:acc: 0.897781,loss:0.227570
[epoch:53,batch:719]:acc: 0.898307,loss:0.226995
[epoch:53,batch:749]:acc: 0.898250,loss:0.227481
[epoch:53,batch:779]:acc: 0.898277,loss:0.227543
[epoch:53,batch:809]:acc: 0.898110,loss:0.227905
[epoch:53,batch:839]:acc: 0.898512,loss:0.227403
[epoch:53,batch:869]:acc: 0.898994,loss:0.226460
[epoch:53,batch:899]:acc: 0.899097,loss:0.226655
[epoch:53,batch:899]: val_loss:0.371307,val_acc:0.866931,val_total:4539
[epoch:53,batch:929]:acc: 0.899194,loss:0.226330
[epoch:53,batch:959]:acc: 0.899609,loss:0.226614
[epoch:53,batch:989]:acc: 0.899463,loss:0.226517
[epoch:53] :acc: 0.899423,loss:0.226543,lr:0.000000,patience:0
[epoch:53

[epoch:57,batch:659]:acc: 0.899290,loss:0.228575
[epoch:57,batch:689]:acc: 0.898732,loss:0.229107
[epoch:57,batch:719]:acc: 0.898958,loss:0.228862
[epoch:57,batch:749]:acc: 0.899208,loss:0.229104
[epoch:57,batch:779]:acc: 0.899399,loss:0.227638
[epoch:57,batch:809]:acc: 0.900077,loss:0.226442
[epoch:57,batch:839]:acc: 0.900000,loss:0.227131
[epoch:57,batch:869]:acc: 0.900180,loss:0.227065
[epoch:57,batch:899]:acc: 0.900035,loss:0.226940
[epoch:57,batch:899]: val_loss:0.373043,val_acc:0.866490,val_total:4539
[epoch:57,batch:929]:acc: 0.900403,loss:0.226192
[epoch:57,batch:959]:acc: 0.900488,loss:0.226895
[epoch:57,batch:989]:acc: 0.900852,loss:0.226326
[epoch:57] :acc: 0.900873,loss:0.226240,lr:0.000000,patience:0
[epoch:57]: val_loss:0.376289,val_acc:0.865829,
Epoch 58/59
----------
[epoch:58,batch:29]:acc: 0.895833,loss:0.204090
[epoch:58,batch:59]:acc: 0.901563,loss:0.214382
[epoch:58,batch:89]:acc: 0.902083,loss:0.226717
[epoch:58,batch:119]:acc: 0.903125,loss:0.227198
[epoch:58,bat

In [4]:
def TrainWithRawData(path,epochNum):
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet101/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'ResNet101', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        patience=0
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'ResNet101', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet101', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet101', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/ResNet101/2018-11-04_acc_best.pth',80)

[+] loading model... Done
min_loss is :0.338854
val_correct is 0.874862
Epoch 57/79
----------
begin lr is  0.0001
[epoch:57,batch:29]:acc: 0.918750,loss:0.181818
[epoch:57,batch:59]:acc: 0.917188,loss:0.189465
[epoch:57,batch:89]:acc: 0.911806,loss:0.200851
[epoch:57,batch:119]:acc: 0.911458,loss:0.204693
[epoch:57,batch:149]:acc: 0.911875,loss:0.205010
[epoch:57,batch:179]:acc: 0.909375,loss:0.206568
[epoch:57,batch:209]:acc: 0.910417,loss:0.205255
[epoch:57,batch:239]:acc: 0.915104,loss:0.199309
[epoch:57,batch:269]:acc: 0.914583,loss:0.200440
[epoch:57,batch:299]:acc: 0.913646,loss:0.201988
[epoch:57,batch:299]: val_loss:0.335713,val_acc:0.874422,val_total:4539
save new model loss,now loss is  0.33571332693099976
[epoch:57,batch:329]:acc: 0.913826,loss:0.202941
[epoch:57,batch:359]:acc: 0.913542,loss:0.203607
[epoch:57,batch:389]:acc: 0.913462,loss:0.203016
[epoch:57,batch:419]:acc: 0.913170,loss:0.204922
[epoch:57,batch:449]:acc: 0.913611,loss:0.204118
[epoch:57,batch:479]:acc: 0.

[epoch:61,batch:119]:acc: 0.914323,loss:0.196920
[epoch:61,batch:149]:acc: 0.914167,loss:0.194948
[epoch:61,batch:179]:acc: 0.917014,loss:0.191329
[epoch:61,batch:209]:acc: 0.917857,loss:0.190793
[epoch:61,batch:239]:acc: 0.917969,loss:0.192629
[epoch:61,batch:269]:acc: 0.918171,loss:0.192859
[epoch:61,batch:299]:acc: 0.917604,loss:0.192974
[epoch:61,batch:299]: val_loss:0.335300,val_acc:0.872879,val_total:4539
save new model loss,now loss is  0.3352998197078705
[epoch:61,batch:329]:acc: 0.918182,loss:0.192208
[epoch:61,batch:359]:acc: 0.917622,loss:0.194324
[epoch:61,batch:389]:acc: 0.917308,loss:0.195814
[epoch:61,batch:419]:acc: 0.917932,loss:0.195552
[epoch:61,batch:449]:acc: 0.916736,loss:0.196946
[epoch:61,batch:479]:acc: 0.916992,loss:0.196335
[epoch:61,batch:509]:acc: 0.915870,loss:0.198578
[epoch:61,batch:539]:acc: 0.916493,loss:0.197063
[epoch:61,batch:569]:acc: 0.916667,loss:0.196785
[epoch:61,batch:599]:acc: 0.916146,loss:0.197774
[epoch:61,batch:599]: val_loss:0.335484,val

[epoch:65,batch:239]:acc: 0.918490,loss:0.194284
[epoch:65,batch:269]:acc: 0.917824,loss:0.193285
[epoch:65,batch:299]:acc: 0.917604,loss:0.193005
[epoch:65,batch:299]: val_loss:0.337259,val_acc:0.873981,val_total:4539
[epoch:65,batch:329]:acc: 0.915814,loss:0.195026
[epoch:65,batch:359]:acc: 0.916580,loss:0.194742
[epoch:65,batch:389]:acc: 0.915465,loss:0.195532
[epoch:65,batch:419]:acc: 0.915997,loss:0.194227
[epoch:65,batch:449]:acc: 0.915972,loss:0.194665
[epoch:65,batch:479]:acc: 0.916276,loss:0.194318
[epoch:65,batch:509]:acc: 0.916299,loss:0.194516
[epoch:65,batch:539]:acc: 0.915451,loss:0.195316
[epoch:65,batch:569]:acc: 0.916064,loss:0.195363
[epoch:65,batch:599]:acc: 0.915729,loss:0.196238
[epoch:65,batch:599]: val_loss:0.337294,val_acc:0.873761,val_total:4539
[epoch:65,batch:629]:acc: 0.915675,loss:0.196335
[epoch:65,batch:659]:acc: 0.915388,loss:0.196330
[epoch:65,batch:689]:acc: 0.914402,loss:0.197406
[epoch:65,batch:719]:acc: 0.913932,loss:0.198197
[epoch:65,batch:749]:ac